# Interval Calculations from Attributes Data

Based on NCSLI Recommended Pracitce 1. Methods A3 and S2 are implemented in the `TestInterval` and `BinomialInterval` classes, respectively. The other methods described in RP-1 were not implemented because they are "not recommended but remain documented in this RP to discourage its “reinvention” and maintain awareness of the drawbacks of similar methods."

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from suncal.intervals import BinomialInterval, TestInterval, BinomialIntervalAssets, TestIntervalAssets

## Method S2 - Binomial Method

This method uses the observed measurement reliability as a function of time between calibrations, fit to several different reliability models, to determine the best interval. Use this method if calibrations have been made at many different intervals.

Data may be entered as time-since-calibraiton vs. reliability using `BinomialInterval()` or as calibration time vs. pass/fail for individual assets using `BinomialIntervalAssets()`.

Running `.calculate()` will compute all the reliability models, assign a figure of merit to each, and return the interval resulting from the best model.

In [2]:
# Reliability data from Table D-1 in RP1
ti = [4,7,10,13,21,28,40,48]           # Weeks between calibrations
ni = np.array([4,6,14,13,22,49,18,6])  # Number of calibrations in each interval of ti
Ri = [1.0, .83333, .6429, .6154, .5455, .4082, .5000, .3333]    # Observed measurement reliability

In [3]:
# Data is already summarized terms of reliability. Use BinomialInterval().
intv = BinomialInterval(Rtarget=.75, ti=ti, Ri=Ri, ni=ni)
intv.calculate()

## Best Fit Model


Interval   | Model        | Rejection Confidence   | 95.0% Confidence Interval Range  
----------|------------|----------------------|---------------------------------
9.0        | Random Walk  | 7.0%                   | 6.3 - 11.2

All results are stored in the `.out` parameter of the `BinomialInterval` object. To see results of every reliability model, use `.report_all()`. Other calculation results can be accessed through the `.out` parameter as well.

In [4]:
intv.out.report_all()

Reliability Model   | Interval   | Rejection Confidence   | F-Test    | Figure of Merit  
-------------------|----------|----------------------|---------|-----------------
Random Walk         | 9          | 7.04%                  | True      | 196.74           
Restricted Walk     | 8          | 7.54%                  | True      | 178.53           
Mortality Drift     | 8          | 10.55%                 | True      | 127.55           
Log Normal          | 9          | 15.33%                 | True      | 90.39            
Mixed Exponential   | 8          | 15.37%                 | True      | 87.52            
Weibull             | 9          | 22.50%                 | True      | 61.58            
Warranty            | 8          | 59.55%                 | True      | 22.59            
Drift               | 8          | 70.51%                 | True      | 19.08            
Exponential         | 11         | 26.58%                 | True      | 6.85             
Modified Gamma      | 19         | 99.30%                 | False     | 2.10             


![IMG0][]



[IMG0]: 

In [5]:
intv.out.interval  # Best interval as float

9.0

In [6]:
intv.out.best  # Name of the best model

'Random Walk'

### Data as individual calibrations

If the data is in terms of calibration date and pass/fail, rather than summarized reliability vs interval values, use the `BinomailIntervalAssets` class. The `updateasset` method can be called with calibration dates and pass/fail values on an individual asset. Data from multiple assets can be combined using different asset names in `updateasset`. The data will be converted into time-betwee-calibrations vs observed measurement reliability by binning into ten intervals (by default).

In [7]:
# Make up some data based on RP-1 table D-1. Results won't be the same because RP1 used hand-selected bin edges.
enddate = [0]
passfail = [1]

def addit(passes, total, weeks):
    for i in range(total):
        enddate.append(enddate[-1] + np.random.uniform(*weeks))
    passfail.extend([1]*passes + [0]*(total-passes))
    
addit(4, 4, (2, 4))
addit(5, 6, (5, 7))
addit(9, 14, (8, 10))
addit(8, 13, (11, 13))
addit(12, 22, (19, 21))
addit(20, 49, (26, 28))
addit(9, 18, (37, 40))
addit(2, 6, (48, 51))

In [8]:
# Calculate the interval
intv = BinomialIntervalAssets(Rt=.8, bins=10)
intv.updateasset('asset123', enddate, passfail)
intv.calculate()
intv.out.report_allmodels()

Reliability Model   | Interval   | Rejection Confidence   | F-Test    | Figure of Merit  
-------------------|----------|----------------------|---------|-----------------
Restricted Walk     | 7          | 7.66%                  | True      | 127.47           
Mortality Drift     | 6          | 8.61%                  | True      | 109.02           
Random Walk         | 7          | 11.66%                 | True      | 83.71            
Mixed Exponential   | 5          | 17.80%                 | True      | 50.39            
Weibull             | 5          | 27.44%                 | True      | 32.70            
Exponential         | 9          | 46.47%                 | True      | 22.36            
Warranty            | 0          | 100.00%                | False     | 0.00             
Drift               | 0          | 100.00%                | False     | 0.00             
Log Normal          | 0          | 100.00%                | False     | 0.00             
Modified Gamma      | 18         | 99.84%                 | False     | 2.06

---

## Method A3 - Test Interval Method

Use Method A3 if all calibrations were done at nearly the same interval. It calculates a new interval based on observed reliability of all calibrations done under the same interval.

If the data is a list of actual pass/fail status for each DUT, use the `TestIntervalAssets` class to set up the calculation.

Here, y is a list of actual pass/fail (1/0) values on DUTs of the same class calibrated in nearly the same interval of 365 days.
(Data from 2019 NCSLI Symposium Tutorial on Intervals).

In [9]:
# Assuming all values in y were measured at the assigned interval (I0=365)
# Use TestInterval class with number in-tolerance and total number as inputs

y = np.array([1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,])  # Pass/fail values
intv = TestInterval(intol=np.count_nonzero(y), n=len(y), I0=365, Rt=.95)  # Existing interval I0=365 days.
intv.calculate()

Parameter                              | Value            
--------------------------------------|-----------------
Suggested Interval                     | 231         
Calculated Interval                    | 231         
Current Interval Rejection Confidence  | 99.49%           
True reliability range                 | 65.16% - 82.86%  
Observed Reliability                   | 75.00% (15 / 20)

In [10]:
# If calibration dates are given, can use them to ensure all calibration intervals were close
# to the assigned interval I0. Some may be discarded if the actual interval was too long or short.
# The last line in output table lists how many calibrations were discarded.

startdates = np.arange(0, 365*(len(y)), step=365)
caldates = np.arange(365, 365*(len(y)+1), step=365)  # Just make all cal intervals exactly 365
intv = TestIntervalAssets(I0=365, Rt=.95)
intv.updateasset('asset123', startdates=startdates, enddates=caldates, passfail=y)
intv.calculate()

Parameter                               | Value            
---------------------------------------|-----------------
Suggested Interval                      | 231         
Calculated Interval                     | 231         
Current Interval Rejection Confidence   | 99.49%           
True reliability range                  | 65.16% - 82.86%  
Observed Reliability                    | 75.00% (15 / 20) 
Rejected calibrations (wrong interval)  | 0